In [ ]:
#| default_exp chatgpt

In [ ]:
#| export
import re
from mytools.paths import *
from mytools.tools import *
import openai
openai.api_key = get_api_key(api_path,'openai')

In [ ]:
#| export
from transformers import CLIPProcessor, CLIPModel
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
tokenizer = processor.tokenizer
import pandas as pd

In [ ]:
#| export
def ask_chatgpt(messages,text,return_request=False,model="gpt-3.5-turbo-0301"):
    request = {"role": "user", "content": text}
    messages.append(request)
    r = openai.ChatCompletion.create(
      model=model,  #"gpt-4", #"gpt-4" "gpt-3.5-turbo"
      messages=messages
    ).to_dict()
    
    responce = r['choices'][0].to_dict()['message'].to_dict()
    print (responce['content'])
    if return_request: return responce,request 
    else: return responce




In [ ]:
messages = [
    {"role": "system", "content": 'test'},
    {"role": "assistant", "content": 'test'},
           ]

In [ ]:
ask_chatgpt(messages,'test')

Is there anything specific you would like me to test or answer for you?


{'role': 'assistant',
 'content': 'Is there anything specific you would like me to test or answer for you?'}

In [ ]:
messages

[{'role': 'system', 'content': 'test'},
 {'role': 'assistant', 'content': 'test'},
 {'role': 'user', 'content': 'test'}]

In [ ]:
#| export
def crop_tokens(messages,max_tokens=1000):
    if len(messages)==0: return []
    tokens_len = [len(tokenizer(m['content'])['input_ids']) for m in messages]
    token_count_df = pd.DataFrame(messages)
    token_count_df['len'] = [len(tokenizer(m['content'])['input_ids']) for m in messages]
    token_count_df = token_count_df.drop_duplicates('content').sort_index(ascending=False)
    token_count_df = token_count_df[token_count_df['len'].cumsum()<max_tokens]
    token_index = token_count_df.sort_index().index
    messages = messages[token_index[0]:token_index[-1]]
    return messages

In [ ]:
#| export
def strip_text(text,urls):
    for url in urls: text = text.replace(url,'').strip()
    while True:
        start_len = len(text)
        text = text.replace('  ',' ')
        text = text.replace('\n \n','\n')
        stop_len = len(text)
        if start_len == stop_len: break
        
    return text
    

In [ ]:
#| export
extract_urls = lambda text: re.findall('https?://[^\s]+', text)

get_button_text = lambda url:url.split('q=')[1].split('&')[0].replace('+',' ')

In [ ]:
!nbdev_export